**Website used in this project: https://www.thetvdb.com/series/the-ellen-degeneres-show**, 

**I do not own the data**(but code is definitely mine). This is for educational purpose and a project to learn web scraping using selenium.

In [13]:
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from IPython.display import Image
import datetime

**Adding Chrome Driver path**

In [2]:
PATH ="C:\Program Files\chromedriver.exe"

**Cell below will open chrome browser**

In [3]:
driver = webdriver.Chrome(PATH)

**Adding Link of the website we want to scrape from**

In [4]:
driver.get('https://www.thetvdb.com/series/the-ellen-degeneres-show')

# Final scrapper code: 
**Do keep the chrome browser launched by this notebook open, in front of your eyes, you will see the magic of selenium and python**

In [5]:
Date = []
Season_ep = []
Guests = []


for i in range(3,21):
    
    element=WebDriverWait(driver,20).until(EC.visibility_of_element_located((By.XPATH ,'//*[@id="page-series"]/div[3]/div[2]/div[2]/div[6]/div[1]/ul/li['+str(i)+']/h4/a')))
    element.click()
    
    for j in range(1,200):
        try:
            
            Season_ep.append(driver.find_element_by_xpath('//*[@id="page-season"]/div[4]/div[2]/div[1]/table/tbody/tr['+str(j)+']/td[1]').text)
            Guests.append(driver.find_element_by_xpath('//*[@id="page-season"]/div[4]/div[2]/div[1]/table/tbody/tr['+str(j)+']/td[2]/a').text)
            Date.append(driver.find_element_by_xpath('//*[@id="page-season"]/div[4]/div[2]/div[1]/table/tbody/tr['+str(j)+']/td[3]/div').text)
        
        except NoSuchElementException:
            
            continue  
    print('Season:'+str(i-2)+' done') 
    
    element=WebDriverWait(driver,20).until(EC.visibility_of_element_located((By.XPATH ,'//*[@id="page-season"]/div[3]/div/a[3]')))
    element.click()

Season:1 done
Season:2 done
Season:3 done
Season:4 done
Season:5 done
Season:6 done
Season:7 done
Season:8 done
Season:9 done
Season:10 done
Season:11 done
Season:12 done
Season:13 done
Season:14 done
Season:15 done
Season:16 done
Season:17 done
Season:18 done


In [6]:
len(Date),len(Guests),len(Season_ep) #checking if each column has same size of data

(3091, 3091, 3091)

**Clubbing everything together and making a dataframe**

In [7]:
d = zip(Season_ep,Date,Guests)
mapped = list(d)
df = pd.DataFrame(mapped, columns =['Season', 'Date','Guests'])

In [10]:
df

,Season,Date,Guests
0,S01E01,"September 8, 2003",Series Premiere
1,S01E02,"September 9, 2003",Justin Timberlake
2,S01E03,"September 10, 2003",Betty White/Sherri Shepherd
3,S01E04,"September 11, 2003",Megan Mullally
4,S01E05,"September 12, 2003",Dennis Quaid/Lisa Marie Presley
...,...,...,...
3086,S18E174,"June 28, 2021",Guest Host Tiffany Haddish with Tyrese Gibson ...
3087,S18E175,"June 29, 2021","Guest Host Chelsea Handler with Kristen Bell, ..."
3088,S18E176,"June 30, 2021","Guest Host Wanda Sykes with Don Cheadle, Ilana..."
3089,S18E177,"July 1, 2021",Guest Host Chelsea Handler with Robin Wright a...


**Let's verify the dataset, if it is accurate or not** 

So I remember **Deepika Padukone** went to Ellen show. If this is complete database as I believe. It should contain that entry. Lets also see who were **other guests that day**.

In [11]:
df[df['Guests'].str.contains('Deepika Padukone')]

,Season,Date,Guests
2270,S14E84,"January 18, 2017","Matt LeBlanc, Deepika Padukone"


Season 14 Episode 84, in January 18 2017, It was around release of her Hollywood debut movie **XXX:Return of Xander Cage** with Vin Diesel.

In [14]:
#Converting date to a much favourable format

df['Date'] = df['Date'].apply(lambda x: datetime.datetime.strptime(x, '%B %d, %Y').date())
df['Date'] = pd.to_datetime(df['Date'])

In [16]:
df.tail(10)

,Season,Date,Guests
3081,S18E169,2021-06-21,"Cecily Strong, Ann Dowd"
3082,S18E170,2021-06-22,"Chris “Ludacris” Bridges, James TW"
3083,S18E171,2021-06-23,"Henry Golding, Damien Caillaud, Urbain Ngobobo..."
3084,S18E172,2021-06-24,"Minnie Driver, Blake Shelton"
3085,S18E173,2021-06-25,Guest Host Stephen “tWitch” Boss with Casey Wi...
3086,S18E174,2021-06-28,Guest Host Tiffany Haddish with Tyrese Gibson ...
3087,S18E175,2021-06-29,"Guest Host Chelsea Handler with Kristen Bell, ..."
3088,S18E176,2021-06-30,"Guest Host Wanda Sykes with Don Cheadle, Ilana..."
3089,S18E177,2021-07-01,Guest Host Chelsea Handler with Robin Wright a...
3090,S18E178,2021-07-02,Guest Host Stephen “tWitch” Boss with Jay Phar...


In [ ]:
df.to_csv('Ellen_dataset(2003-2021).csv')